# 京东JData算法大赛(1)-数据清洗

  比赛的题目是高潜用户的购买意向的预测,从机器学习的角度来讲我们可以认为这是一个二分类的任务.那么我们就是尝试去构建自己的正负样本.  
  由于我们拿到的是原始数据,里面存在很多噪声,因而第一步我们先要对数据清洗,比如说:  
- 去掉只有购买记录的用户(没有可用的历史浏览等记录来预测用户将来的购买意向)  
- 去掉浏览量很大而购买量很少的用户(惰性用户或爬虫用户)
- 去掉最后5(7)天没有记录(交互)的商品和用户
- ......

为了能够进行上述清洗,在此首先构造了简单的用户(user)行为特征和商品(item)行为行为特征,对应于两张表user_table和item_table
- **user_table**特征包括:  
  user_id(用户id),age(年龄),sex(性别),  
  user_lv_cd(用户级别),browse_num(浏览数),  
  addcart_num(加购数),delcart_num(删购数),  
  buy_num(购买数),favor_num(收藏数),  
  click_num(点击数),buy_addcart_ratio(购买加购转化率),  
  buy_browse_ratio(购买浏览转化率),  
  buy_click_ratio(购买点击转化率),  
  buy_favor_ratio(购买收藏转化率)  
- **item_table**特征包括:  
  sku_id(商品id),attr1,attr2,  
  attr3,cate,brand,browse_num,  
  addcart_num,delcart_num,  
  buy_num,favor_num,click_num,  
  buy_addcart_ratio,buy_browse_ratio,  
  buy_click_ratio,buy_favor_ratio,  
  comment_num(评论数),  
  has_bad_comment(是否有差评),  
  bad_comment_rate(差评率)

接下来我们将采用python2.7, pandas等工具简单介绍如何实现上述过程.

In [3]:
# 定义文件名
ACTION_201602_FILE = "data/JData_Action_201602.csv"
ACTION_201603_FILE = "data/JData_Action_201603.csv"
ACTION_201603_EXTRA_FILE = "data/JData_Action_201603_extra.csv"
ACTION_201604_FILE = "data/JData_Action_201604.csv"
COMMENT_FILE = "data/JData_Comment.csv"
PRODUCT_FILE = "data/JData_Product.csv"
USER_FILE = "data/JData_User.csv"
NEW_USER_FILE = "data/JData_User_New.csv"
USER_TABLE_FILE = "data/user_table.csv"
ITEM_TABLE_FILE = "data/item_table.csv"

In [4]:
# 导入相关包
import pandas as pd
import numpy as np
from collections import Counter

一般为了快速验证自己的想法我们会从原数据中抽取一部分作为样本，在此每个文件我将抽取10000行记录．  
我将原始文件放在data_ori/文件夹下，样本文件放在data/下．

In [3]:
num_sample = 10000
file_list = [ACTION_201602_FILE, ACTION_201603_FILE,
             ACTION_201603_EXTRA_FILE, ACTION_201604_FILE,
             NEW_USER_FILE, COMMENT_FILE]

for fname in file_list:
    with open("data_ori/" + fname[5:], 'rb') as fi:
        with open(fname, 'wb') as fo:
            for i in xrange(num_sample):
                fo.write(fi.readline())

接下来以构建user_table为例,item_table与之类似.

In [4]:
# 功能函数: 对每一个user分组的数据进行统计
def add_type_count(group):
    behavior_type = group.type.astype(int)
    # 用户行为类别
    type_cnt = Counter(behavior_type)
    # 1: 浏览 2: 加购 3: 删除
    # 4: 购买 5: 收藏 6: 点击
    group['browse_num'] = type_cnt[1]
    group['addcart_num'] = type_cnt[2]
    group['delcart_num'] = type_cnt[3]
    group['buy_num'] = type_cnt[4]
    group['favor_num'] = type_cnt[5]
    group['click_num'] = type_cnt[6]

    return group[['user_id', 'browse_num', 'addcart_num',
                  'delcart_num', 'buy_num', 'favor_num',
                  'click_num']]

由于用户行为数据量较大,一次性读入可能造成内存错误(Memory Error),因而使用pandas的分块(chunk)读取.

In [5]:
#　对action数据进行统计
# 根据自己的需求调节chunk_size大小
def get_from_action_data(fname, chunk_size=100000):
    reader = pd.read_csv(fname, header=0, iterator=True)
    chunks = []
    loop = True
    while loop:
        try:
            # 只读取user_id和type两个字段
            chunk = reader.get_chunk(chunk_size)[["user_id", "type"]]
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped")
    # 将块拼接为pandas dataframe格式
    df_ac = pd.concat(chunks, ignore_index=True)
    # 按user_id分组，对每一组进行统计
    df_ac = df_ac.groupby(['user_id'], as_index=False).apply(add_type_count)
    # 将重复的行丢弃
    df_ac = df_ac.drop_duplicates('user_id')

    return df_ac

In [6]:
# 将各个action数据的统计量进行聚合
def merge_action_data():
    df_ac = []
    df_ac.append(get_from_action_data(fname=ACTION_201602_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201603_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201603_EXTRA_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201604_FILE))

    df_ac = pd.concat(df_ac, ignore_index=True)
    # 用户在不同action表中统计量求和
    df_ac = df_ac.groupby(['user_id'], as_index=False).sum()
    #　构造转化率字段
    df_ac['buy_addcart_ratio'] = df_ac['buy_num'] / df_ac['addcart_num']
    df_ac['buy_browse_ratio'] = df_ac['buy_num'] / df_ac['browse_num']
    df_ac['buy_click_ratio'] = df_ac['buy_num'] / df_ac['click_num']
    df_ac['buy_favor_ratio'] = df_ac['buy_num'] / df_ac['favor_num']
    
    # 将大于１的转化率字段置为１(100%)
    df_ac.ix[df_ac['buy_addcart_ratio'] > 1., 'buy_addcart_ratio'] = 1.
    df_ac.ix[df_ac['buy_browse_ratio'] > 1., 'buy_browse_ratio'] = 1.
    df_ac.ix[df_ac['buy_click_ratio'] > 1., 'buy_click_ratio'] = 1.
    df_ac.ix[df_ac['buy_favor_ratio'] > 1., 'buy_favor_ratio'] = 1.

    return df_ac

In [7]:
#　从JData_User表中抽取需要的字段
def get_from_jdata_user():
    df_usr = pd.read_csv(NEW_USER_FILE, header=0)
    df_usr = df_usr[["user_id", "age", "sex", "user_lv_cd"]]
    return df_usr

构建user table的main函数

In [8]:
user_base = get_from_jdata_user()
user_behavior = merge_action_data()

# 连接成一张表，类似于SQL的左连接(left join)
user_behavior = pd.merge(user_base, user_behavior, on=['user_id'], how='left')
# 保存为user_table.csv
user_behavior.to_csv(USER_TABLE_FILE, index=False)

Iteration is stopped
Iteration is stopped
Iteration is stopped
Iteration is stopped


item table的构建过程与user table的类似，这里直接运行我事先写好的脚本．

In [9]:
!python create_item_table.py

Iteration is stopped
Iteration is stopped
Iteration is stopped
Iteration is stopped


### 数据清洗

In [5]:
df_usr = pd.read_csv(USER_TABLE_FILE, header=0)

# 输出前5行数据
df_usr.head(5)

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
0,54,-1,2,1,102.0,1.0,0.0,0.0,0.0,228.0,0.000000,0.000000,0.000000,NaN
1,79,3,2,2,674.0,20.0,15.0,0.0,11.0,1075.0,0.000000,0.000000,0.000000,0.00
2,100,1,2,3,310.0,5.0,2.0,3.0,4.0,479.0,0.600000,0.009677,0.006263,0.75
3,266,0,2,2,264.0,0.0,0.0,0.0,0.0,515.0,NaN,0.000000,0.000000,NaN
4,333,-1,2,4,300.0,46.0,0.0,2.0,1.0,730.0,0.043478,0.006667,0.002740,1.00


In [6]:
# 输出购买数为0的后5条数据
df_usr[df_usr['buy_num'] == 0].tail(5)


,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
102812,81130,2,0,5,9.0,0.0,0.0,0.0,0.0,13.0,NaN,0.0,0.0,NaN
102942,44318,3,0,5,230.0,11.0,3.0,0.0,0.0,369.0,0.0,0.0,0.0,NaN
103155,53654,-1,2,5,56.0,0.0,0.0,0.0,0.0,95.0,NaN,0.0,0.0,NaN
103462,19566,3,0,5,6.0,0.0,2.0,0.0,0.0,9.0,NaN,0.0,0.0,NaN
103563,75712,2,0,5,77.0,1.0,1.0,0.0,1.0,173.0,0.0,0.0,0.0,0.0


In [7]:
# 删除购买数为0的用户
df_usr = df_usr[df_usr['buy_num'] != 0]

In [8]:
# 输出格式设置，保留三位小数
pd.options.display.float_format = '{:,.3f}'.format

# 输出user table的统计信息,包括
# 总数，均值，方差，最小值，1/4分位数，1/2分位数，3/4分位数，最大值
df_usr.describe()

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
count,"31,235.000","31,235.000","31,235.000","31,235.000","31,184.000","31,184.000","31,184.000","31,184.000","31,184.000","31,184.000","31,184.000","31,184.000","31,184.000","31,184.000"
mean,"50,151.790",2.055,0.970,4.348,433.447,13.698,6.377,1.688,2.359,701.879,0.310,0.015,0.025,0.825
std,"29,580.226",1.220,0.958,0.768,557.339,20.063,11.318,1.561,6.894,911.288,0.305,0.035,0.125,0.317
min,3.000,-1.000,0.000,1.000,1.000,0.000,0.000,1.000,0.000,0.000,0.001,0.000,0.000,0.002
25%,"24,612.500",2.000,0.000,4.000,106.000,3.000,1.000,1.000,0.000,161.000,0.091,0.003,0.002,1.000
50%,"48,847.000",2.000,1.000,5.000,259.000,8.000,3.000,1.000,0.000,406.000,0.200,0.006,0.004,1.000
75%,"75,959.500",3.000,2.000,5.000,551.250,17.000,8.000,2.000,2.000,893.000,0.429,0.013,0.009,1.000
max,"103,611.000",5.000,2.000,5.000,"16,038.000",907.000,746.000,51.000,538.000,"16,979.000",1.000,1.000,1.000,1.000


In [23]:
# 查看购买数为１而浏览数大于8000的用户
df_usr[(df_usr['buy_num'] < 2) & (df_usr['browse_num'] > 6000)]

,user_id,age,sex,user_lv_cd,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num,buy_addcart_ratio,buy_browse_ratio,buy_click_ratio,buy_favor_ratio
51840,19313,2,0,5,"9,214.000",191.000,61.000,1.000,1.000,"10,952.000",0.005,0.000,0.000,1.000
58350,72063,3,2,4,"6,304.000",79.000,50.000,1.000,19.000,"8,503.000",0.013,0.000,0.000,0.053
